In [1]:
import at
import sys
from at import Param, ParamArray
if sys.version_info.minor < 9:
    from importlib_resources import files, as_file
else:
    from importlib.resources import files, as_file

In [2]:
fname = 'hmba.mat'
with as_file(files('machine_data') / fname) as path:
    ring = at.load_lattice(path)

# Parameters

Parameters are objects of class {py:class}`.Param` which can be used instead of numeric values as {py:class}`.Element` attributes.

Parameters are initialised with a **scalar** numeric value. They have an optional name, only used to identify them in printed output:

In [3]:
total_length = Param(2.5, name='total length')
dlength = Param(1.0)
print(total_length, dlength)

Param(2.5, name='total length') Param(1.0, name='param2')


The value of a parameter can be read or modified through its {py:attr}`~.variables.Variable.value` property. {py:meth}`~.variables.Variable.set` and {py:meth}`~.variables.Variable.get` methods are also available:

In [4]:
print(total_length.value)
total_length.value = 2.4
print(total_length)
total_length.set(2.3)
total_length.get()

2.5
Param(2.4, name='total length')


2.3

Arithmetic combinations of parameters create new read-only parameters of class {py:class}`.ParamBase`, whose value is permanently kept up-to-date:

In [5]:
qlength = total_length - dlength
print(qlength)
dlength.value = 0.9
print(qlength)

ParamBase(1.2999999999999998, name='calc1')
ParamBase(1.4, name='calc1')


Parameters may be assigned to {py:class}`.Element` attributes, for instance on initialisation:

In [6]:
dr1 = at.Drift('DR1', dlength)
qf1 = at.Quadrupole('QF1', qlength, 0.6)
print(dr1)
print(qf1)

Drift:
	FamName : DR1
	Length : Param(0.9, name='param2')
	PassMethod : DriftPass
Quadrupole:
	FamName : QF1
	Length : ParamBase(1.4, name='calc1')
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : 10
	MaxOrder : 1
	PolynomA : [0.0 0.0]
	PolynomB : [0.0 0.6]
	K : 0.6


The {py:class}`.Element` attributes keep their type so that all the processing of elements either in python functions or in C integrators is unchanged:

In [7]:
print(dr1.Length, type(dr1.Length))

0.9 <class 'float'>


## Assigning parameters

### To a single element
Parameters may be assigned to {py:class}`.Element` attributes in several ways:

**At element creation:**

In [8]:
dr2 = at.Drift('DR2', dlength)

**By converting a numeric attribute into a parameter:**

In [9]:
qd1 = at.Quadrupole('QD1', 0.5, -0.4)
ql = qd1.parametrise('Length')
print(ql)

Param(0.5, name='param3')


**By normal assignment:**

In [10]:
qstrength = Param(-0.5, name='quad strength')
qd1.Length = Param(0.2)
qd1.PolynomB[1] = qstrength
print(qd1)

Quadrupole:
	FamName : QD1
	Length : Param(0.2, name='param5')
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : 10
	MaxOrder : 1
	PolynomA : [0.0 0.0]
	PolynomB : [0.0 Param(-0.5, name='quad strength')]
	K : Param(-0.5, name='quad strength')


**With the {py:meth}`~.Element.set_parameter` method:**

In [11]:
qd1.set_parameter('Length', qlength)
print(qd1)

Quadrupole:
	FamName : QD1
	Length : ParamBase(1.4, name='calc1')
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : 10
	MaxOrder : 1
	PolynomA : [0.0 0.0]
	PolynomB : [0.0 Param(-0.5, name='quad strength')]
	K : Param(-0.5, name='quad strength')


### To selected elements of a {py:class}`.Lattice`
To act on several elements in a single step, {py:class}`.Lattice` methods similar to {py:class}`.Element` methods are available

**Convert numeric attributes into parameters:**

The attribute of all the selected elements is replaced by a single parameter whose initial value is the average of the original values.

In [12]:
kf1 = ring.parametrise('QF1[AE]', 'PolynomB', index=1, name='kf1')
print(kf1)
print(ring[5])

Param(2.5394599781303304, name='kf1')
Quadrupole:
	FamName : QF1A
	Length : 0.311896
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : 20
	FringeQuadEntrance : 1
	FringeQuadExit : 1
	MaxOrder : 1
	PolynomA : [0 0]
	PolynomB : [0.0 Param(2.5394599781303304, name='kf1')]
	K : Param(2.5394599781303304, name='kf1')


**Use the {py:meth}`~.Lattice.set_parameter` method:**

The attribute of all the selected elements is replaced by the provided parameter,

In [13]:
lf1 = Param(0.311896, name='lf1')
ring.set_parameter('QF1[AE]', 'Length', lf1)
print(ring[117])

Quadrupole:
	FamName : QF1E
	Length : Param(0.311896, name='lf1')
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : 20
	FringeQuadEntrance : 1
	FringeQuadExit : 1
	MaxOrder : 1
	PolynomA : [0 0]
	PolynomB : [0.0 Param(2.5394599781303304, name='kf1')]
	K : Param(2.5394599781303304, name='kf1')


Once a Parameter is assigned to an attribute, it acquires the type and the constraints of
the attribute. For instance:

In [14]:
num_int_steps = Param(14.4)
ring.set_parameter(at.Multipole, 'NumIntSteps', num_int_steps)
print(ring[5].NumIntSteps, type(ring[5].NumIntSteps))

14 <class 'int'>


In [15]:
num_int_steps.value = -5

ValueError: Value must be greater of equal to 0

The parameter behaves as the attribute.

## Retrieving parameters

Since the values of {py:class}`.Element` attributes keep their original type, they cannot be used to access the underlying parameter. The only way to retrieve it is to use the {py:meth}`~.Element.get_parameter` method. a {py:obj}`TypeError` is raised if the attribute is not
a Parameter.

In [16]:
ql = qd1.get_parameter('Length')
print(ql)
print("ql is qlength:", ql is qlength)

ParamBase(1.4, name='calc1')
ql is qlength: True


This also works for items in array attributes:

In [17]:
qs = qd1.get_parameter('PolynomB', index=1)
print(qs)
print("qs is qstrength:", qs is qstrength)

Param(-0.5, name='quad strength')
qs is qstrength: True


## Checking parametrisation

The {py:meth}`~.Element.is_parametrised` method may be applied to:
- a full element: it returns true is any of its attributes is a parameter,
- an array attribute: it returns true if any of its items is a parameter,
- a scalar attribute or an item of an array.

In [18]:
print(qd1.is_parametrised())
print(qd1.is_parametrised('PolynomB', index=1))
print(qd1.is_parametrised('PolynomB', index=0))

True
True
False


## Removing parameters
Removing the parameters will "freeze" the element at its current value. The {py:meth}`~.Element.unparametrise` method is defined for both {py:class}`.Element` and {py:class}`.Lattice`, and may be applied to:
- a full element: all the parameters are replaced by their value,
- an array attribute: the whole {py:class}`.ParamArray` is replaced by a numpy array,
- a scalar attribute or an item of an array.

### In an Element

In [19]:
print(qd1)
qd1.unparametrise('Length')
print(qd1)
qd1.unparametrise('PolynomB', 1)
print(qd1)

Quadrupole:
	FamName : QD1
	Length : ParamBase(1.4, name='calc1')
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : 10
	MaxOrder : 1
	PolynomA : [0.0 0.0]
	PolynomB : [0.0 Param(-0.5, name='quad strength')]
	K : Param(-0.5, name='quad strength')
Quadrupole:
	FamName : QD1
	Length : 1.4
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : 10
	MaxOrder : 1
	PolynomA : [0.0 0.0]
	PolynomB : [0.0 Param(-0.5, name='quad strength')]
	K : Param(-0.5, name='quad strength')
Quadrupole:
	FamName : QD1
	Length : 1.4
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : 10
	MaxOrder : 1
	PolynomA : [0.0 0.0]
	PolynomB : [0.0 -0.5]
	K : -0.5


### In a Lattice

In [20]:
print(ring[5])
ring.unparametrise('QF1[AE]', 'PolynomB', index=1)
print(ring[5])
ring.unparametrise('QF1[AE]')
print(ring[117])

Quadrupole:
	FamName : QF1A
	Length : Param(0.311896, name='lf1')
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : Param(14, name='param8')
	FringeQuadEntrance : 1
	FringeQuadExit : 1
	MaxOrder : 1
	PolynomA : [0 0]
	PolynomB : [0.0 Param(2.5394599781303304, name='kf1')]
	K : Param(2.5394599781303304, name='kf1')
Quadrupole:
	FamName : QF1A
	Length : Param(0.311896, name='lf1')
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : Param(14, name='param8')
	FringeQuadEntrance : 1
	FringeQuadExit : 1
	MaxOrder : 1
	PolynomA : [0 0]
	PolynomB : [0.0 2.5394599781303304]
	K : 2.5394599781303304
Quadrupole:
	FamName : QF1E
	Length : 0.311896
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : 14
	FringeQuadEntrance : 1
	FringeQuadExit : 1
	MaxOrder : 1
	PolynomA : [0 0]
	PolynomB : [0.0 2.5394599781303304]
	K : 2.5394599781303304


## Parameter history
Parameter values are kept in an {py:attr}`~.variables.Variable.history` buffer. The properties {py:attr}`~.variables.Variable.initial_value`, {py:attr}`~.variables.Variable.last_value` and {py:attr}`~.variables.Variable.previous_value` are also available:

In [21]:
dlength.value = 1.1
print(dlength.history)
print(dlength.initial_value)
print(dlength.previous_value)

[1.0, 0.9, 1.1]
1.0
0.9


After varying parameters, in matching for instance, the current status can be printed:

In [22]:
print(dlength.status())


        Name      Initial          Final        Variation

      param2    1.000000e+00    1.100000e+00    1.000000e-01


Parameters may be reset to a previous history value with the {py:meth}`~.variables.Variable.set_initial` and {py:meth}`~.variables.Variable.set_previous` methods. The history is shortened accordingly.

In [23]:
dlength.set_previous()
print(dlength, dlength.history)
dlength.set_initial()
print(dlength, dlength.history)

Param(0.9, name='param2') [1.0, 0.9]
Param(1.0, name='param2') [1.0]


## Copying and Saving

In a shallow copy of an {py:class}`.Element`, the parameter attributes are shared between the {py:class}`.Element` and its copy.

In a deep copy of an {py:class}`.Element`, a copy of the original parameter is set in the {py:class}`.Element` copy.

When saving a lattice with parametrised elements, as a `.mat`, a `.m` or `.repr` file, all parametrisation is removed, and a "frozen" state of the lattice is saved.

In pickle dumps of an {py:class}`.Element`, parameters are preserved.